# Imports, install and mount

<!--  -->

In [1]:
import torch
from torch_geometric.data import Data
import pandas as pd
import pickle
from torch_geometric.transforms import RandomLinkSplit

# Settings


In [2]:
file_path = "/home/ebutz/ESL2024/data/iric.tsv"

altail_mapping_save_path = "/home/ebutz/ESL2024/data/altailed_mapped_iric.pickle"
altail_dict_save_path = "/home/ebutz/ESL2024/data/altail_DICT_iric_DICT.pickle"
datasets_save_path = '/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/little_dataset_'
val_path = datasets_save_path + 'VAL'
test_path = datasets_save_path + 'TEST'
train_path = datasets_save_path + 'TRAIN'

device = 'cpu'

## Reading and mapping graph

In [3]:
iric = pd.read_csv(file_path, delimiter='\t')
display(iric)

,subject,predicate,object
0,GO:0000001,is_a,GO:0048311
1,GO:0000001,is_a,GO:0048308
2,GO:0000002,is_a,GO:0007005
3,GO:0000003,is_a,GO:0008150
4,GO:0000006,is_a,GO:0005385
...,...,...,...
1452516,OsNippo12g255000,interacts_with,OsNippo07g025800
1452517,OsNippo12g255000,interacts_with,OsNippo07g207000
1452518,OsNippo12g255000,interacts_with,OsNippo07g207600
1452519,OsNippo12g255000,interacts_with,OsNippo10g150350


In [22]:
iric.groupby(['object'])

In [4]:
iric = pd.read_csv(file_path, delimiter='\t')

# Mapping entities and relations to ids
entity_set = set(iric['object']).union(set(iric['subject']))
entity_to_mapping = {entity: int(i) for i, entity in enumerate(entity_set)}
relation_set = set(iric['predicate'])
relation_to_mapping = {relation: int(i) for i, relation in enumerate(relation_set)}

iric['mapped_subject'] = iric['subject'].apply(lambda x: entity_to_mapping[x])
iric['mapped_predicate'] = iric['predicate'].apply(lambda x: relation_to_mapping[x])
iric['mapped_object'] = iric['object'].apply(lambda x: entity_to_mapping[x])
print(iric.dtypes)
display(iric)

subject             object
predicate           object
object              object
mapped_subject       int64
mapped_predicate     int64
mapped_object        int64
dtype: object


,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object
0,GO:0000001,is_a,GO:0048311,49100,0,35069
1,GO:0000001,is_a,GO:0048308,49100,0,26571
2,GO:0000002,is_a,GO:0007005,9015,0,26957
3,GO:0000003,is_a,GO:0008150,379,0,11514
4,GO:0000006,is_a,GO:0005385,78507,0,48466
...,...,...,...,...,...,...
1452516,OsNippo12g255000,interacts_with,OsNippo07g025800,75202,3,57885
1452517,OsNippo12g255000,interacts_with,OsNippo07g207000,75202,3,48300
1452518,OsNippo12g255000,interacts_with,OsNippo07g207600,75202,3,42235
1452519,OsNippo12g255000,interacts_with,OsNippo10g150350,75202,3,33420


In [6]:
from tqdm import tqdm
tqdm.pandas()

iric['mapped_alt_tails'] = iric.progress_apply(
                                             lambda row: 
                                             iric.loc[(iric['mapped_subject'] == row['mapped_subject']) & 
                                                      (iric['mapped_predicate'] == row['mapped_predicate']) & 
                                                      (iric['mapped_object'] != row['mapped_object']), 
                                                                                                               'mapped_object'].values, 
                                             axis=1)


display(iric)
iric.to_pickle(mapping_save_path)
print(f"Saved mapping at {mapping_save_path} :")
display(pd.read_pickle(mapping_save_path))

,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object,mapped_alt_tails
0,GO:0000001,is_a,GO:0048311,49100,0,35069,[26571]
1,GO:0000001,is_a,GO:0048308,49100,0,26571,[35069]
2,GO:0000002,is_a,GO:0007005,9015,0,26957,[]
3,GO:0000003,is_a,GO:0008150,379,0,11514,[]
4,GO:0000006,is_a,GO:0005385,78507,0,48466,[]
...,...,...,...,...,...,...,...
1452516,OsNippo12g255000,interacts_with,OsNippo07g025800,75202,3,57885,"[13863, 19116, 15480, 48300, 42235, 33420, 56119]"
1452517,OsNippo12g255000,interacts_with,OsNippo07g207000,75202,3,48300,"[13863, 19116, 15480, 57885, 42235, 33420, 56119]"
1452518,OsNippo12g255000,interacts_with,OsNippo07g207600,75202,3,42235,"[13863, 19116, 15480, 57885, 48300, 33420, 56119]"
1452519,OsNippo12g255000,interacts_with,OsNippo10g150350,75202,3,33420,"[13863, 19116, 15480, 57885, 48300, 42235, 56119]"


Saved mapping at /home/ebutz/ESL2024/data/altailed_mapped_iric.pickle:


,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object,mapped_alt_tails
0,GO:0000001,is_a,GO:0048311,49100,0,35069,[26571]
1,GO:0000001,is_a,GO:0048308,49100,0,26571,[35069]
2,GO:0000002,is_a,GO:0007005,9015,0,26957,[]
3,GO:0000003,is_a,GO:0008150,379,0,11514,[]
4,GO:0000006,is_a,GO:0005385,78507,0,48466,[]
...,...,...,...,...,...,...,...
1452516,OsNippo12g255000,interacts_with,OsNippo07g025800,75202,3,57885,"[13863, 19116, 15480, 48300, 42235, 33420, 56119]"
1452517,OsNippo12g255000,interacts_with,OsNippo07g207000,75202,3,48300,"[13863, 19116, 15480, 57885, 42235, 33420, 56119]"
1452518,OsNippo12g255000,interacts_with,OsNippo07g207600,75202,3,42235,"[13863, 19116, 15480, 57885, 48300, 33420, 56119]"
1452519,OsNippo12g255000,interacts_with,OsNippo10g150350,75202,3,33420,"[13863, 19116, 15480, 57885, 48300, 42235, 56119]"


In [ ]:
def update_alt_tails(row, mapped_alt_tails):
    key = (row['mapped_subject'], row['mapped_predicate'])
    if key not in mapped_alt_tails:
        try:
            mapped_alt_tails[key] = set(row['mapped_alt_tails'])
        except Exception as e:
            print(f"Problem with {row['mapped_alt_tails']}: {e}")
    else:
        try:
            mapped_alt_tails[key].update(set(row['mapped_alt_tails']))
        except Exception as e:
            print(f"Problem with {row['mapped_alt_tails']}: {e}")

mapped_alt_tails = {}
tqdm.pandas(desc="Making dict of alternatives tails")
iric.progress_apply(lambda row: update_alt_tails(row, mapped_alt_tails), axis=1)

print('First key-value pair in dict :', list(mapped_alt_tails.items())[0])

for key, value in mapped_alt_tails.items():
    mapped_alt_tails[key]=list(value)

with open('filename.pickle', 'wb') as handle:
    pickle.dump(mapped_alt_tails, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Building init vars for Data :

In [4]:
# Edges index
heads = list(iric['mapped_subject'])
tails = list(iric['mapped_object'])
edge_index = torch.tensor([heads,tails], dtype=torch.long)
# edges states
edge_attributes = torch.tensor(iric['mapped_predicate'])

iric_pyg = Data(
                num_nodes = len(entity_set),
                edge_index = edge_index,
                edge_attr = edge_attributes
                )

print(iric_pyg)

print("\nDataset looks valid ? \n",iric_pyg.validate(raise_on_error=True))

Data(edge_index=[2, 10001], edge_attr=[10001], num_nodes=3343)

Dataset looks valid ? 
 True


## Splitting dataset

In [5]:
transform = RandomLinkSplit(
                            num_val = 0.1,
                            num_test = 0.1,
                            is_undirected=False,
                            add_negative_train_samples=False,
                            )

train, val, test = transform(iric_pyg)

torch.save(obj=train, f = train_path)
torch.save(obj=test, f = test_path)
torch.save(obj=val, f = val_path)

print(f'test saved at {test_path}\nval saved at : {val_path}\ntrain saved at : {train_path}\n')

print('Loaded datasets look valid (val, test, train):',
torch.load(val_path).validate(raise_on_error=True),
torch.load(test_path).validate(raise_on_error=True),
torch.load(train_path).validate(raise_on_error=True),'\n')

print('Before :', val)
print(' After :', torch.load(val_path))

test saved at /home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/little_dataset_TEST.pickle
val saved at : /home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/little_dataset_VAL.pickle
train saved at : /home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/little_dataset_TRAIN.pickle

Loaded datasets look valid (val, test, train): True True True 

Before : Data(edge_index=[2, 8001], edge_attr=[8001], num_nodes=3343, edge_label=[2000], edge_label_index=[2, 2000])
 After : Data(edge_index=[2, 8001], edge_attr=[8001], num_nodes=3343, edge_label=[2000], edge_label_index=[2, 2000])
